# **Phenological diversity with Sentinel 2**

### <span style="color:red">   Import packages</span>

In [1]:
import ee
ee.Initialize()
import geemap
Map = geemap.Map()

In [2]:
import pandas as pd

### <span style="color:red">  Load collections</span>

In [19]:
# forest mask from ESA CCI (unchanged forests)
forestmask = ee.Image('users/marcogirardello/phenoutils/mask_unchanged_500m')
# entire collection from Sentinel 2
collectionS2 = ee.ImageCollection("COPERNICUS/S2_SR")
# sampling areas
samples = ee.FeatureCollection('users/marcogirardello/exploratoryproject/samples')

### <span style="color:red">   Required functions</span>

In [20]:
def maskCloudAndShadows(image):
    cloudProb = image.select('MSK_CLDPRB')
    snowProb = image.select('MSK_SNWPRB')
    cloud = cloudProb.lt(5)
    snow = snowProb.lt(5)
    scl = image.select('SCL')
    shadow = scl.eq(3) # 3 = cloud shadow
    cirrus = scl.eq(10) # 10 = cirrus
    # cloud probability less than 5% or cloud shadow classification
    mask = (cloud.And(snow)).And(cirrus.neq(1)).And(shadow.neq(1))
    return image.updateMask(mask) 

# add dn for product aggregated into 16 days
def add_dn_date(img,beginDate=None,n=None,IncludeYear=False):
    if beginDate is None:
        beginDate = img.get('system:time_start')
    else:
        beginDate = beginDate
    if IncludeYear is False:
        IncludeYear = True
    if n is None:
        n = 8
    beginDate = ee.Date(beginDate)
    year  = beginDate.get('year')
    month = beginDate.get('month')
    diff  = beginDate.difference(ee.Date.fromYMD(year, 1, 1), 'day').add(1)
    dn    = diff.subtract(1).divide(n).floor().add(1).int()
    yearstr  = year.format('%d') 
    dn = dn.format('%02d')
    return ee.Image(img).set('system:time_start', beginDate.millis()).set('date', beginDate.format('yyyy-MM-dd')).set('Year', yearstr).set('Month',beginDate.format('MM')).set('YearMonth', beginDate.format('YYYY-MM')).set('dn', dn)

# wrapper function for add_dn_date
def add_dn_date_all(Year,days):
    def wrapper(image0):
        tmp = add_dn_date(img = image0,IncludeYear=Year,n = days)
        return tmp
    return (wrapper)

def smoothing(image):
    collection = ee.ImageCollection.fromImages(image.get('images'))
    return ee.Image(image).addBands(collection.mean())

### <span style="color:red">  Filter random square</span>

In [7]:
onesq = samples.filterMetadata('polyID','equals',33)

### <span style="color:red"> Filter collection and add NDVI text</span>

In [17]:
res = collectionS2.\
    filterBounds(onesq).\
    filterDate('2017-03-28','2020-12-31').filterMetadata('CLOUDY_PIXEL_PERCENTAGE','less_than',30)#.\
    #map(maskCloudAndShadows)#.\
    #map(lambda image: image.normalizedDifference(['B8', 'B4']).rename('NDVI'))#.select('NDVI')

In [21]:
res.first().propertyNames().getInfo()

['DATATAKE_IDENTIFIER',
 'AOT_RETRIEVAL_ACCURACY',
 'CLOUD_COVERAGE_PERCENTAGE',
 'SPACECRAFT_NAME',
 'FORMAT_CORRECTNESS_FLAG',
 'SATURATED_DEFECTIVE_PIXEL_PERCENTAGE',
 'system:id',
 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B8A',
 'CLOUD_SHADOW_PERCENTAGE',
 'MEAN_SOLAR_AZIMUTH_ANGLE',
 'system:footprint',
 'VEGETATION_PERCENTAGE',
 'SOLAR_IRRADIANCE_B12',
 'system:version',
 'SOLAR_IRRADIANCE_B10',
 'SOLAR_IRRADIANCE_B11',
 'GENERATION_TIME',
 'SOLAR_IRRADIANCE_B8A',
 'SENSOR_QUALITY_FLAG',
 'CLOUD_COVERAGE_ASSESSMENT',
 'THIN_CIRRUS_PERCENTAGE',
 'system:time_end',
 'WATER_VAPOUR_RETRIEVAL_ACCURACY',
 'system:time_start',
 'DATASTRIP_ID',
 'PROCESSING_BASELINE',
 'SENSING_ORBIT_NUMBER',
 'GEOMETRIC_QUALITY_FLAG',
 'NODATA_PIXEL_PERCENTAGE',
 'SENSING_ORBIT_DIRECTION',
 'GRANULE_ID',
 'LOW_PROBA_CLOUDS_PERCENTAGE',
 'REFLECTANCE_CONVERSION_CORRECTION',
 'MEDIUM_PROBA_CLOUDS_PERCENTAGE',
 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B8',
 'DATATAKE_TYPE',
 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B9',
 'MEAN_INCIDENCE_A

#### Add day

In [149]:
# add day 
collection = collectionS2.map(add_dn_date_all(Year = False, days = 16))

#### Aggregate data into 16-day composites

In [115]:
tmpseas = ["%02d" % x for x in list(range(1, 23+1))]
tmpseas1 = ee.List(tmpseas)

In [119]:
res.select('B8').projection().getInfo()

{'type': 'Projection',
 'crs': 'EPSG:32734',
 'transform': [10, 0, 199980, 0, -10, 9600040]}

In [128]:
res1 = collectionS2.\
    filterBounds(onesq).\
    filterDate('2017-03-28','2020-12-31').filterMetadata('CLOUDY_PIXEL_PERCENTAGE','less_than',30).\
    map(maskCloudAndShadows).median()

In [124]:
img = res.max()
img1 = img.setDefaultProjection(crs = 'EPSG:32734',scale = 10).reduceResolution(ee.Reducer.stdDev(), bestEffort = False, maxPixels = 200).reproject(ee.Projection('EPSG:32734').scale(100, 100)).updateMask(1) 

In [126]:
visualization = {
  'min': 0.0,
  'max': 0.3,
  'bands': ['B4', 'B3', 'B2'],
}

In [132]:
Map.addLayer(res1, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2500, 'gamma': 1.1},
                'S2 cloud-free mosaic')

In [84]:
Map.addLayer(img1,'','stdvv')

In [125]:
task = ee.batch.Export.image.toAsset(image = img1,description = 's2test_max_africa',
                                     assetId='users/marcogirardello/exploratoryproject/s2test_max_africa',
                                     crs = 'EPSG:32734',scale = 10,maxPixels=1e13,region=onesq.geometry())
task.start()

In [ ]:
img.reduceRegion()

In [111]:
Map = geemap.Map()
#Map.set_center(-67.4484,-7.9461,9)
Map.set_center(19.6385, -3.1962,9)
Map


Map(center=[-3.1962, 19.6385], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButto…

#### Check temporal coverage

In [18]:
# calculate total number of observations and filter by forest cover
countS2 = s2_2019.select(0).count().updateMask(forestmask.eq(1))

In [14]:
#Map.addLayer(countS2,{'min':1,'max':200,'palette':['blue','green','yellow','red']},'S2 count filtered')

In [19]:
#Map

#### Aggregate data 16-day 